<a href="https://colab.research.google.com/github/FatemeZamanian/DeepLearning/blob/main/Flowers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from matplotlib import pyplot as plt
import numpy as np
import os
import datetime

In [ ]:
dataset_path="/content/drive/MyDrive/Flowers"
width=height=224
batch_size=32
data_generator=ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    rotation_range=10,
)